In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Sử dụng class MyPCA

In [2]:
class MyPCA:
    def __init__(self, n_components=None):
        """
        n_components: số thành phần chính muốn giữ lại
        Nếu None thì giữ toàn bộ
        """
        self.n_components = n_components

    def fit(self, X):
        """
        X: mảng numpy shape (n_samples, n_features)
        Tính:
         - self.components_: ma trận (n_components, n_features)
         - self.explained_variance_: vector các trị riêng (n_components,)
         - self.explained_variance_ratio_: EVR (n_components,)
         - self.cumulative_explained_variance_: CEVR (n_components,)
        """
        # 1) Tâm dữ liệu
        self.mean_ = np.mean(X, axis=0)
        Xc = X - self.mean_
        # 2) Ma trận hiệp phương sai
        C = np.cov(Xc, rowvar=False)
        # 3) Tính trị riêng
        eigvals, eigvecs = np.linalg.eigh(C)
        # 4) Sắp xếp giảm dần
        idx = np.argsort(eigvals)[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]
        # 5) Chọn số thành phần
        if self.n_components is not None:
            eigvals = eigvals[:self.n_components]
            eigvecs = eigvecs[:, :self.n_components]
        # 6) Lưu kết quả
        self.components_ = eigvecs.T
        self.explained_variance_ = eigvals
        total_var = np.sum(eigvals)
        self.explained_variance_ratio_ = eigvals / total_var
        self.cumulative_explained_variance_ = np.cumsum(self.explained_variance_ratio_)
        return self

    def transform(self, X):
        """
        Chiếu X lên không gian thành phần chính đã fit trước đó
        """
        Xc = X - self.mean_
        return Xc.dot(self.components_.T)

    def fit_transform(self, X):
        return self.fit(X).transform(X)

# 2. Chuẩn bị dữ liệu

In [18]:
df = pd.read_csv('../data/ABIDE2.csv')

In [19]:
# Giả sử các cột ['site','subject','group'] không dùng cho clustering
X_abide = df.drop(columns=['site','subject','group']).values
y_abide = df['group'].values 

# Chuẩn hóa
X_abide_std = (X_abide - X_abide.mean(axis=0)) / X_abide.std(axis=0)

In [20]:
# Xác định số PC
k = 10

In [21]:
# Giảm chiều với PCA, lấy ví dụ k PC đầu
pca_abide = MyPCA(n_components=k)
X10_abide = pca_abide.fit_transform(X_abide_std)

# 3. Phân cụm KMeans 

In [22]:
def kmeans(X, k=2, max_iters=100, tol=1e-6):
    """
    X: (n_samples, n_features)
    k: số cluster
    Trả về:
      labels: (n_samples,) nhãn cluster
      centers: (k, n_features)
    """
    n, d = X.shape
    # 1) Khởi tạo centers bằng cách chọn k điểm ngẫu nhiên
    rng = np.random.default_rng(seed=42)
    centers = X[rng.choice(n, size=k, replace=False)].copy()
    labels = np.zeros(n, dtype=int)

    for it in range(max_iters):
        # 2) Gán nhãn
        dist = np.linalg.norm(X[:, None, :] - centers[None, :, :], axis=2)
        new_labels = np.argmin(dist, axis=1)

        # 3) Cập nhật centers
        new_centers = np.zeros_like(centers)
        for j in range(k):
            pts = X[new_labels==j]
            # Nếu không có điểm, giữ nguyên center cũ
            new_centers[j] = pts.mean(axis=0) if len(pts)>0 else centers[j]

        # Kiểm tra hội tụ
        shift = np.linalg.norm(new_centers - centers)
        centers = new_centers
        labels = new_labels
        if shift < tol:
            break

    return labels, centers

In [23]:
labels_abide, centers_abide = kmeans(X10_abide, k=2)

# 4. Đánh giá

In [24]:
# Chuyển y_abide ("Cancer"/"Normal") thành nhãn số {0,1}
mapping = {'Normal':0, 'Cancer':1}
y_num = np.vectorize(mapping.get)(y_abide)

# Xác định map cụm->nhãn sao cho accuracy cao nhất
# Thử cả hai gán: cluster0->0 hoặc 1
acc0 = np.mean(labels_abide == y_num)
acc1 = np.mean((1-labels_abide) == y_num)
if acc1 > acc0:
    labels_abide = 1 - labels_abide
accuracy = np.mean(labels_abide == y_num)
print(f"Accuracy clustering: {accuracy*100:.2f}%")

# Ma trận nhầm lẫn
from collections import Counter
# Tính TP, TN, FP, FN
TP = np.sum((labels_abide==1)&(y_num==1))
TN = np.sum((labels_abide==0)&(y_num==0))
FP = np.sum((labels_abide==1)&(y_num==0))
FN = np.sum((labels_abide==0)&(y_num==1))
print("Confusion matrix:")
print(f"          Pred=0     Pred=1")
print(f"True=0    {TN:>8d}   {FP:>8d}")
print(f"True=1    {FN:>8d}   {TP:>8d}")
# Precision, recall
precision = TP / (TP+FP) if TP+FP>0 else 0
recall    = TP / (TP+FN) if TP+FN>0 else 0
print(f"Precision: {precision*100:.2f}%")
print(f"Recall:    {recall*100:.2f}%")

Accuracy clustering: 53.98%
Confusion matrix:
          Pred=0     Pred=1
True=0         353        188
True=1         274        189
Precision: 50.13%
Recall:    40.82%


In [25]:
# Silhouette score
from sklearn.metrics import silhouette_score
sil = silhouette_score(X10_abide, labels_abide)
print(f"Silhouette score: {sil:.3f}")

Silhouette score: 0.287
